In [ ]:
#We start by installing LDA
!pip install lda

     |████████████████████████████████| 351 kB 7.4 MB/s 
     |████████████████████████████████| 99 kB 10.9 MB/s 


In [ ]:
#And making the relevant imports
import numpy as np
import lda
import pandas as pd

In [ ]:
# Load the data
all_data_df = pd.read_csv('11534_tweets.csv') #This file must have ID
# Print head
all_data_df.head()

,id,name,screen_name,created_at,text,location,followers_count,friends_count,listed_count,retweet_count,favorite_count
0,1460631062023917570,Lucas Stollmeier,StollmeierLucas,2021-11-16 15:29:17,b'RT @SheldonEvans: Just hit 200K!\n\xf0\x9f\x...,NaN,0,22,0,4702,0
1,1460631061038198796,علی حیدری,lyhydry1,2021-11-16 15:29:16,b'RT @heliosarmy: @lexfridman We believe that ...,NaN,3,3,0,84,0
2,1460631060778303490,Coldstar,Coldsta93422887,2021-11-16 15:29:16,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,5,346,1,1318,0
3,1460631059889115149,PJ,Jose_1017,2021-11-16 15:29:16,b'@davidgokhshtein Load your bags we\xe2\x80\x...,Puerto Rico,55,581,2,0,0
4,1460631057636679680,ام سوسو,omsosoo0,2021-11-16 15:29:16,b'RT @Next100xToken: \xf0\x9f\x94\xa5 #Next100...,NaN,84,4340,0,520,0


In [ ]:
communities_df = pd.read_csv('communities.csv')

In [ ]:
communities_df

,User,Community
0,00sabbir,1
1,0Iktier,1
2,0f4ZVDzolSjPOy8,1
3,12gods1,1
4,1354Sahand,1
...,...,...
4876,yticmaerd,3
4877,yujaemcityx,3
4878,yuryorl,3
4879,zackdrury1988,3


In [ ]:
def find_messages_with_str(df, *argv):
  """This function takes a dataframe = df, and a any number of word parameters 
  through the *argv arguments. This function returns a NEW dataframe with the
  messages that contain all the words (word1 & word2 & ... wordn) unwrapped 
  from the *argvs

  Parameters:
         df = Dataframe with the columns ['Date', 'Messages']
      *argv = Every argument is a different word to be matched
  """

  #Create a regular expression pattern containing all words
  regex_pattern = ''
  for arg in argv:
    regex_pattern += '(?=.*' + arg + ')'

  #filter the dataframe through the regular expression and return the new df
  result = df['text'].str.contains(pat = regex_pattern, regex=True)
  filtered_df = df.loc[result]
  return filtered_df

def find_messages_with_retweet(df):
  # Create a regular expression pattern containing all words
  regex_pattern = 'RT @'

  # filter the dataframe through the regular expression and return the new df
  result = df['text'].str.contains(pat = regex_pattern, regex=True)
  filtered_df = df.loc[result]
  return filtered_df

In [ ]:
attention_df = find_messages_with_retweet(all_data_df)
attention_df

,id,name,screen_name,created_at,text,location,followers_count,friends_count,listed_count,retweet_count,favorite_count
0,1460631062023917570,Lucas Stollmeier,StollmeierLucas,2021-11-16 15:29:17,b'RT @SheldonEvans: Just hit 200K!\n\xf0\x9f\x...,NaN,0,22,0,4702,0
1,1460631061038198796,علی حیدری,lyhydry1,2021-11-16 15:29:16,b'RT @heliosarmy: @lexfridman We believe that ...,NaN,3,3,0,84,0
2,1460631060778303490,Coldstar,Coldsta93422887,2021-11-16 15:29:16,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,5,346,1,1318,0
4,1460631057636679680,ام سوسو,omsosoo0,2021-11-16 15:29:16,b'RT @Next100xToken: \xf0\x9f\x94\xa5 #Next100...,NaN,84,4340,0,520,0
6,1460631055405182982,yuu,menang_yu88,2021-11-16 15:29:15,b'RT @ShiborgToken: #SHIBORGARMY are you excit...,NaN,536,3649,1,589,0
...,...,...,...,...,...,...,...,...,...,...,...
11528,1461103727356616707,りるる🍌✨💎🚀🏰,riruru2777,2021-11-17 22:47:29,b'RT @giveawaybanana: BANANA GIVEAWAY #70\n\nP...,NaN,433,1506,7,56,0
11529,1461103726064812032,Muhammad Asad Nadeem,Muhammad8255011,2021-11-17 22:47:28,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,7,111,0,18092,0
11530,1461103722126323716,Benedito Campos,Benedit80592865,2021-11-17 22:47:28,b'RT @AIBCsummit: A huge thank you to everyone...,NaN,26,63,1,1544,0
11531,1461103721388167172,Yen Ngoc,YenNgoc45,2021-11-17 22:47:27,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,0,22,0,4495,0


In [ ]:
attention_df['original_user'] = attention_df['text'].str.extract(r'RT @([^\s]+)')
attention_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,name,screen_name,created_at,text,location,followers_count,friends_count,listed_count,retweet_count,favorite_count,original_user
0,1460631062023917570,Lucas Stollmeier,StollmeierLucas,2021-11-16 15:29:17,b'RT @SheldonEvans: Just hit 200K!\n\xf0\x9f\x...,NaN,0,22,0,4702,0,SheldonEvans:
1,1460631061038198796,علی حیدری,lyhydry1,2021-11-16 15:29:16,b'RT @heliosarmy: @lexfridman We believe that ...,NaN,3,3,0,84,0,heliosarmy:
2,1460631060778303490,Coldstar,Coldsta93422887,2021-11-16 15:29:16,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,5,346,1,1318,0,CryptoTownEU:
4,1460631057636679680,ام سوسو,omsosoo0,2021-11-16 15:29:16,b'RT @Next100xToken: \xf0\x9f\x94\xa5 #Next100...,NaN,84,4340,0,520,0,Next100xToken:
6,1460631055405182982,yuu,menang_yu88,2021-11-16 15:29:15,b'RT @ShiborgToken: #SHIBORGARMY are you excit...,NaN,536,3649,1,589,0,ShiborgToken:
...,...,...,...,...,...,...,...,...,...,...,...,...
11528,1461103727356616707,りるる🍌✨💎🚀🏰,riruru2777,2021-11-17 22:47:29,b'RT @giveawaybanana: BANANA GIVEAWAY #70\n\nP...,NaN,433,1506,7,56,0,giveawaybanana:
11529,1461103726064812032,Muhammad Asad Nadeem,Muhammad8255011,2021-11-17 22:47:28,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,7,111,0,18092,0,CryptoTownEU:
11530,1461103722126323716,Benedito Campos,Benedit80592865,2021-11-17 22:47:28,b'RT @AIBCsummit: A huge thank you to everyone...,NaN,26,63,1,1544,0,AIBCsummit:
11531,1461103721388167172,Yen Ngoc,YenNgoc45,2021-11-17 22:47:27,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,0,22,0,4495,0,CryptoTownEU:


In [ ]:
#Remove the colons 
attention_df['original_user'] = attention_df['original_user'].str.extract('([^\s]+):')
attention_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,name,screen_name,created_at,text,location,followers_count,friends_count,listed_count,retweet_count,favorite_count,original_user
0,1460631062023917570,Lucas Stollmeier,StollmeierLucas,2021-11-16 15:29:17,b'RT @SheldonEvans: Just hit 200K!\n\xf0\x9f\x...,NaN,0,22,0,4702,0,SheldonEvans
1,1460631061038198796,علی حیدری,lyhydry1,2021-11-16 15:29:16,b'RT @heliosarmy: @lexfridman We believe that ...,NaN,3,3,0,84,0,heliosarmy
2,1460631060778303490,Coldstar,Coldsta93422887,2021-11-16 15:29:16,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,5,346,1,1318,0,CryptoTownEU
4,1460631057636679680,ام سوسو,omsosoo0,2021-11-16 15:29:16,b'RT @Next100xToken: \xf0\x9f\x94\xa5 #Next100...,NaN,84,4340,0,520,0,Next100xToken
6,1460631055405182982,yuu,menang_yu88,2021-11-16 15:29:15,b'RT @ShiborgToken: #SHIBORGARMY are you excit...,NaN,536,3649,1,589,0,ShiborgToken
...,...,...,...,...,...,...,...,...,...,...,...,...
11528,1461103727356616707,りるる🍌✨💎🚀🏰,riruru2777,2021-11-17 22:47:29,b'RT @giveawaybanana: BANANA GIVEAWAY #70\n\nP...,NaN,433,1506,7,56,0,giveawaybanana
11529,1461103726064812032,Muhammad Asad Nadeem,Muhammad8255011,2021-11-17 22:47:28,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,7,111,0,18092,0,CryptoTownEU
11530,1461103722126323716,Benedito Campos,Benedit80592865,2021-11-17 22:47:28,b'RT @AIBCsummit: A huge thank you to everyone...,NaN,26,63,1,1544,0,AIBCsummit
11531,1461103721388167172,Yen Ngoc,YenNgoc45,2021-11-17 22:47:27,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,0,22,0,4495,0,CryptoTownEU


In [ ]:
community_1_df = communities_df[communities_df['Community'] == 1]
community_2_df = communities_df[communities_df['Community'] == 2]
community_3_df = communities_df[communities_df['Community'] == 3]
community_3_df

,User,Community
4437,19Nic89,3
4438,1GoddRich,3
4439,4bandzz,3
4440,91n60,3
4441,AIBCsummit,3
...,...,...
4876,yticmaerd,3
4877,yujaemcityx,3
4878,yuryorl,3
4879,zackdrury1988,3


In [ ]:
community_1_tweets_df = attention_df[attention_df['screen_name'].isin(list(community_1_df['User'].to_numpy()))]
community_1_tweets_df

,id,name,screen_name,created_at,text,location,followers_count,friends_count,listed_count,retweet_count,favorite_count,original_user
2,1460631060778303490,Coldstar,Coldsta93422887,2021-11-16 15:29:16,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,5,346,1,1318,0,CryptoTownEU
8,1460631052863508492,Bintara Yana,bintara_yana,2021-11-16 15:29:14,"b'RT @tsarnetwork: Integrated environment, dra...",NaN,1,30,0,3201,0,tsarnetwork
15,1460631045821255680,Abdur Rahman Haris,Harisabdurr,2021-11-16 15:29:13,b'RT @ICOAnnouncement: \xf0\x9f\x8e\x99 New Pr...,NaN,12,124,0,16609,0,ICOAnnouncement
24,1460631030935666693,abdoo,abdoo28864890,2021-11-16 15:29:09,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,5,200,0,1318,0,CryptoTownEU
27,1460631023432069121,Bangsat,Bangsat74268833,2021-11-16 15:29:07,b'RT @AirdropDet: \xf0\x9f\x94\x8d New #Airdro...,NaN,3,196,0,4278,0,AirdropDet
...,...,...,...,...,...,...,...,...,...,...,...,...
11511,1461103773275856902,oladele bowofola,Oladelemrboss,2021-11-17 22:47:40,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,"Ikeja-lagos, Nigeria",142,632,2,4182,0,CryptoTownEU
11516,1461103756083404805,Hiếu Dung,HiuDung3,2021-11-17 22:47:36,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,27,248,0,18092,0,CryptoTownEU
11521,1461103747677966339,Alireza5885,Alireza58851,2021-11-17 22:47:34,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,28,1214,0,4004,0,CryptoTownEU
11529,1461103726064812032,Muhammad Asad Nadeem,Muhammad8255011,2021-11-17 22:47:28,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,7,111,0,18092,0,CryptoTownEU


In [ ]:
community_2_tweets_df = attention_df[attention_df['screen_name'].isin(list(community_2_df['User'].to_numpy()))]
community_2_tweets_df

,id,name,screen_name,created_at,text,location,followers_count,friends_count,listed_count,retweet_count,favorite_count,original_user
0,1460631062023917570,Lucas Stollmeier,StollmeierLucas,2021-11-16 15:29:17,b'RT @SheldonEvans: Just hit 200K!\n\xf0\x9f\x...,NaN,0,22,0,4702,0,SheldonEvans
4,1460631057636679680,ام سوسو,omsosoo0,2021-11-16 15:29:16,b'RT @Next100xToken: \xf0\x9f\x94\xa5 #Next100...,NaN,84,4340,0,520,0,Next100xToken
6,1460631055405182982,yuu,menang_yu88,2021-11-16 15:29:15,b'RT @ShiborgToken: #SHIBORGARMY are you excit...,NaN,536,3649,1,589,0,ShiborgToken
9,1460631052162977799,🙏🙏🙏,_H_N_D_K_,2021-11-16 15:29:14,b'RT @revolut20: I TOLD YOU GUYS!\nNumber 1 on...,NaN,2506,4584,1,273,0,revolut20
13,1460631049294147586,nonu 🍀🍀🍀,padukacityx,2021-11-16 15:29:14,b'RT @revolut20: I TOLD YOU GUYS!\nNumber 1 on...,🇮🇩,41,1874,0,273,0,revolut20
...,...,...,...,...,...,...,...,...,...,...,...,...
11520,1461103751679418382,chalya,krystalnim,2021-11-17 22:47:35,b'RT @WILDERCRYPTOO: #shillme your top defi pr...,+62,224,2311,0,885,0,WILDERCRYPTOO
11522,1461103743609425924,gise,gisecityx,2021-11-17 22:47:33,b'RT @POODLETOKEN: At $POODL we think that tru...,NaN,111,827,0,4650,0,POODLETOKEN
11524,1461103739792625664,cii | 🍀BISMILLAH AKUN WANGI🍀,strbryci,2021-11-17 22:47:32,b'RT @CryptoAlcoholi: \xf0\x9f\x9a\xa8 I just ...,🍀AKUN SUPER WANGI🍀,345,4922,1,197,0,CryptoAlcoholi
11527,1461103729361489922,Sedat1986,Sedat87761236,2021-11-17 22:47:29,b'RT @gate_io: #Gateio NEW LISTING VOTE @Acent...,NaN,7,128,0,570,0,gate_io


In [ ]:
community_3_tweets_df = attention_df[attention_df['screen_name'].isin(list(community_3_df['User'].to_numpy()))]
community_3_tweets_df

,id,name,screen_name,created_at,text,location,followers_count,friends_count,listed_count,retweet_count,favorite_count,original_user
16,1460631042541465610,Mari Goug,MariGoug,2021-11-16 15:29:12,"b'RT @ethereum_max: What is ""EthereumMax"" expl...","Perth Amboy, NJ",100,238,0,5,0,ethereum_max
47,1460631006873010176,Mello Moon,MelloMoon_37,2021-11-16 15:29:03,"b'RT @ethereum_max: What is ""EthereumMax"" expl...",Moon,457,1091,1,5,0,ethereum_max
59,1460630991001763848,TheCryptoDream,ProminenceLLC,2021-11-16 15:29:00,b'RT @InuKishu: Tag the stores that should acc...,NaN,64,445,2,38,0,InuKishu
85,1460630970802089985,Wisdom,wisemanwiz_,2021-11-16 15:28:55,b'RT @InuKishu: Tag the stores that should acc...,"Houston, TX",7593,6443,0,38,0,InuKishu
100,1460630953676603393,Safemoon❤🙏🙏,Sanjib38867757,2021-11-16 15:28:51,b'RT @InuKishu: Tag the stores that should acc...,NaN,105,192,2,38,0,InuKishu
...,...,...,...,...,...,...,...,...,...,...,...,...
11422,1461103960249491460,Frank the tank,Frank77471275,2021-11-17 22:48:24,b'RT @AIBCsummit: A huge thank you to everyone...,NaN,90,186,1,1544,0,AIBCsummit
11451,1461103890594680841,Zeusxanthi,PanosVarti,2021-11-17 22:48:08,"b""RT @ElongateInform: PARTNERSHIP: #ELONGATE h...",NaN,67,182,0,19,0,ElongateInform
11461,1461103878942998531,saeidabbasi,saeidab39872150,2021-11-17 22:48:05,b'RT @InuKishu: @BigONEexchange @anndylian Tha...,NaN,50,38,2,22,0,InuKishu
11466,1461103875327336457,Nandi Suwandi,nandi4ever,2021-11-17 22:48:04,b'RT @pascal_kardinal: The only thing I can te...,Jakarta,72,77,3,194,0,pascal_kardinal


In [ ]:
for row in community_3_tweets_df['text']:
  print(row)

b'RT @ethereum_max: What is "EthereumMax" explained in more detail; an excerpt from page 8 of the Whitepaper. \n\nClick here to read the Whitep\xe2\x80\xa6'
b'RT @ethereum_max: What is "EthereumMax" explained in more detail; an excerpt from page 8 of the Whitepaper. \n\nClick here to read the Whitep\xe2\x80\xa6'
b'RT @InuKishu: Tag the stores that should accept $KISHU payment! #massadoption \n\n@NOWPayments_io #CRYPTO'
b'RT @InuKishu: Tag the stores that should accept $KISHU payment! #massadoption \n\n@NOWPayments_io #CRYPTO'
b'RT @InuKishu: Tag the stores that should accept $KISHU payment! #massadoption \n\n@NOWPayments_io #CRYPTO'
b'RT @ethereum_max: What is "EthereumMax" explained in more detail; an excerpt from page 8 of the Whitepaper. \n\nClick here to read the Whitep\xe2\x80\xa6'
b'RT @RichQuack: Video of our  $QUACK Billboard Ads on Istanbul! \n#RichQuack #QUACKattackIstanbul #QUACKattackturkey #cryptocurrency #crypto\xe2\x80\xa6'
b'RT @InuKishu: Tag the stores that should acce

In [ ]:
community_1_tweets_df['text'] = community_1_tweets_df['text'].str.lower().replace(r'\\n', ' ', regex=True)
community_2_tweets_df['text'] = community_2_tweets_df['text'].str.lower().replace(r'\\n', ' ', regex=True)
community_3_tweets_df['text'] = community_3_tweets_df['text'].str.lower().replace(r'\\n', ' ', regex=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
!python -m spacy download en

     |████████████████████████████████| 12.0 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
import spacy
sp = spacy.load('en_core_web_sm')
sentence = sp(u'#cryptocurrency \\n\\n\\xf0\\x9f\\x91\\x89 tell you GUYS!\\nNumber')
for word in sentence:
    print(word.text,  word.lemma_)

# #
cryptocurrency cryptocurrency
\n\n\xf0\x9f\x91\x89 \n\n\xf0\x9f\x91\x89
tell tell
you -PRON-
GUYS!\nNumber GUYS!\nNumber


## Community 1 (BTC)

In [ ]:
hashtags = community_1_tweets_df['text'].str.extractall(r'(#\S+)')
from collections import defaultdict
community_1_hashtags = defaultdict(str)
for index,row in hashtags.iterrows():
  print(index[0],row[0])
  community_1_hashtags[index[0]] += row[0].split('\\')[0] + " "

community_1_hashtags_df = pd.DataFrame(community_1_hashtags.items(), columns=['ID','labels'])
community_1_hashtags_df

27 #airdrop:
29 #airdrop:
34 #airdrop:
40 #airdrop:
43 #airdrop:
44 #airdrop:
61 #airdrop:
81 #airdrop:
90 #airdrop:
95 #bitcoin
95 #crypto'
102 #airdrop:
115 #airdrop:
138 #airdrop:
147 #airdrop:
170 #airdrop:
175 #airdrop:
207 #cointigernewlisting
207 #shield/usdt
207 #cointiger
227 #airdrop:
242 #airdrop:
247 #airdrop:
253 #airdrop:
254 #airdrop:
256 #airdrop:
268 #airdrop:
275 #airdrops
275 #crypto'
277 #airdrop:
288 #airdrop:
291 #airdrop:
295 #airdrop:
306 #airdrop:
309 #airdrop:
317 #airdrop:
318 #airdrop:
324 #airdrop:
345 #crypto?'
384 #airdrop:
386 #crypto
387 #airdrop:
394 #airdrop:
397 #airdrop:
401 #fcf
405 #airdrop:
410 #airdrop:
411 #bitcoin
411 #crypto'
414 #airdrop:
420 #airdrop:
422 #airdrop:
460 #cry\xe2\x80\xa6'
482 #hare
490 #airdrop:
493 #airdrop:
496 #hare
518 #airdrop:
528 #hare
548 #dogelon
548 #dogelonmars
548 #elon
548 #doge
548 #crypto
548 #cryptocurrency
549 #airdrop:
552 #hare
558 #airdrop:
565 #airdrop:
582 #haretoken
582 #bsc
582 #bscgem
582 #crypto
585 

,ID,labels
0,27,#airdrop:
1,29,#airdrop:
2,34,#airdrop:
3,40,#airdrop:
4,43,#airdrop:
...,...,...
856,11441,#crypto #bsc
857,11468,#airdrop:
858,11503,#memecontest #crypto
859,11506,#gami #watchtoearn


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = list(community_1_hashtags_df['labels'].to_numpy())

#Vectorize, but consider only words with frequency 3 or more
vectorizer = CountVectorizer(min_df=3)
corpus_vect = vectorizer.fit(corpus)
X = corpus_vect.transform(corpus)

print("Shape of sparse matrix with corpus is:", X.shape)
print("Sample of the values is:\n", X[0:2])

#Vocabulary is already generated by the vectorizer
vocab = tuple(corpus_vect.vocabulary_.keys())

#Titles are the ID's of our instagram posts
titles = tuple(community_1_hashtags_df['ID'].to_numpy())

print("Example of vocabulary:", vocab[0:5])

print("Example of titles:", titles[0:5])

model = lda.LDA(n_topics=2, n_iter=3500, random_state=1)
model.fit(X.toarray())  # Fit the model
topic_word = model.topic_word_ 

INFO:lda:n_documents: 861
INFO:lda:vocab_size: 62
INFO:lda:n_words: 1288
INFO:lda:n_topics: 2
INFO:lda:n_iter: 3500
INFO:lda:<0> log likelihood: -5172
INFO:lda:<10> log likelihood: -4097
INFO:lda:<20> log likelihood: -3954
INFO:lda:<30> log likelihood: -3938
INFO:lda:<40> log likelihood: -3919
INFO:lda:<50> log likelihood: -3946


Shape of sparse matrix with corpus is: (861, 62)
Sample of the values is:
   (0, 1)	1
  (1, 1)	1
Example of vocabulary: ('airdrop', 'bitcoin', 'crypto', 'cointigernewlisting', 'usdt')
Example of titles: (27, 29, 34, 40, 43)


INFO:lda:<60> log likelihood: -3924
INFO:lda:<70> log likelihood: -3924
INFO:lda:<80> log likelihood: -3921
INFO:lda:<90> log likelihood: -3909
INFO:lda:<100> log likelihood: -3933
INFO:lda:<110> log likelihood: -3944
INFO:lda:<120> log likelihood: -3914
INFO:lda:<130> log likelihood: -3911
INFO:lda:<140> log likelihood: -3940
INFO:lda:<150> log likelihood: -3926
INFO:lda:<160> log likelihood: -3919
INFO:lda:<170> log likelihood: -3912
INFO:lda:<180> log likelihood: -3944
INFO:lda:<190> log likelihood: -3922
INFO:lda:<200> log likelihood: -3946
INFO:lda:<210> log likelihood: -3921
INFO:lda:<220> log likelihood: -3908
INFO:lda:<230> log likelihood: -3910
INFO:lda:<240> log likelihood: -3928
INFO:lda:<250> log likelihood: -3918
INFO:lda:<260> log likelihood: -3937
INFO:lda:<270> log likelihood: -3900
INFO:lda:<280> log likelihood: -3910
INFO:lda:<290> log likelihood: -3903
INFO:lda:<300> log likelihood: -3919
INFO:lda:<310> log likelihood: -3901
INFO:lda:<320> log likelihood: -3888
INFO:

In [ ]:
# Print top topic words 
n_top_words = 12
for i, topic_dist in enumerate(topic_word):
     topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
     print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: bitcoin catei dogelon fcf nft cryptonews cryptocurrency trust castorcoin gateio bscgem memecontest
Topic 1: bitcoin catei dogelonmars cryptocurrecy cryptocurrencies binance cointigernewlisting elon babydoge airdrop crypto bullish


In [ ]:
df = pd.DataFrame(topic_word, columns = list(vocab))
#Sort by topic 0
df.transpose().sort_values([0], ascending = False).head(15)

,0,1
bitcoin,0.253271,0.562266
catei,0.133635,0.175041
dogelon,0.052842,0.000015
fcf,0.046627,0.000015
nft,0.045073,0.000015
cryptonews,0.045073,0.000015
cryptocurrency,0.037305,0.000015
trust,0.034197,0.000015
castorcoin,0.034197,0.000015
gateio,0.031090,0.000015


In [ ]:
#Sort by topic 1
df.transpose().sort_values([1], ascending = False).head(15)


,0,1
bitcoin,0.253271,0.562266
catei,0.133635,0.175041
dogelonmars,0.000016,0.054227
cryptocurrecy,0.000016,0.040287
cryptocurrencies,0.000016,0.035640
binance,0.000016,0.015504
cointigernewlisting,0.000016,0.013956
babydoge,0.000016,0.010858
elon,0.000016,0.010858
crypto,0.000016,0.007760


In [ ]:
#Sort by topic 2
df.transpose().sort_values([2], ascending = False).head(15)

,0,1,2,3
catei,0.074077,0.000032,0.314591,0.226849
dogelon,0.000030,0.000032,0.102867,0.000032
fcf,0.000030,0.000032,0.090769,0.000032
nft,0.000030,0.000032,0.087744,0.000032
cryptonews,0.000030,0.000032,0.087744,0.000032
cryptocurrencies,0.000030,0.000032,0.069597,0.000032
castorcoin,0.000030,0.000032,0.066572,0.000032
trust,0.000030,0.000032,0.066572,0.000032
babydoge,0.000030,0.000032,0.012129,0.009753
gami,0.000030,0.000032,0.012129,0.000032


In [ ]:
#Sort by topic 3
df.transpose().sort_values([3], ascending = False).head(15)

,0,1,2,3
bitcoin,0.266602,0.982112,0.000030,0.414782
catei,0.074077,0.000032,0.314591,0.226849
dogelonmars,0.000030,0.000032,0.000030,0.113440
cryptocurrecy,0.000030,0.000032,0.000030,0.084278
cointigernewlisting,0.000030,0.000032,0.000030,0.029194
elon,0.000030,0.000032,0.000030,0.022714
crypto,0.000030,0.000032,0.000030,0.016234
bullish,0.000030,0.000032,0.009104,0.012993
spon,0.000030,0.000032,0.000030,0.009753
btc,0.000030,0.000032,0.000030,0.009753


## Community 2 (Metaverse)

In [ ]:
hashtags = community_2_tweets_df['text'].str.extractall(r'(#\S+)')
from collections import defaultdict
community_2_hashtags = defaultdict(str)
for index,row in hashtags.iterrows():
  print(index[0],row[0])
  community_2_hashtags[index[0]] += row[0].split('\\')[0] + " "

community_2_hashtags_df = pd.DataFrame(community_2_hashtags.items(), columns=['ID','labels'])
community_2_hashtags_df

0 #crypto\xf0\x9f\x8e\x89
4 #next100xtoken
6 #shiborgarmy
17 #crypto\xf0\x9f\x8e\x89
30 #bluezilla's
30 #idos\xf0\x9f\x99\x8c
41 #giveaway
56 #next100xtoken
67 #cryptotrading
67 #cryptoinvestme\xe2\x80\xa6'
75 #giveaway
75 #slex
77 #giveaway\xf0\x9f\x9a\xa8
77 #safepal
77 #crypto
77 #cold\xe2\x80\xa6'
78 #cryptotrading
78 #cryptoinvestme\xe2\x80\xa6'
80 #giveaway
80 #slex
91 #gamefi
92 #next100xtoken
98 #crypto\xf0\x9f\x8e\x89
104 #crypto\xf0\x9f\x8e\x89
106 #next100xtoken
109 #aax's
109 #crypto
131 #shiba
131 #shibarmy
131 #shibainu
131 #shibelon
131 #\xe2\x80\xa6'
139 #nftgiveaway
148 #shiborgarmy
150 #next100xtoken
155 #next100xtoken
157 #next100xtoken
161 #nftgiveaway
161 #nft
161 #nfts
161 #nftcommunity
161 #metaverse
161 #blockchain
161 #ethereum
161 #airdrop
161 #crypto
161 #nf\xe2\x80\xa6'
171 #next100xtoken
172 #cryptotrading
172 #cryptoinvestme\xe2\x80\xa6'
183 #gate.io
187 #next100xtoken
192 #next100xtoken
196 #next100xtoken
200 #giveaway\xf0\x9f\x9a\xa8
200 #safepal
200 #cr

,ID,labels
0,0,#crypto
1,4,#next100xtoken
2,6,#shiborgarmy
3,17,#crypto
4,30,#bluezilla's #idos
...,...,...
1494,11519,#nftgiveaway
1495,11520,#shillme #crypto #defi #shillme #defi #launchp...
1496,11522,#crypto
1497,11524,"#linksync,"


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = list(community_2_hashtags_df['labels'].to_numpy())

#Vectorize, but consider only words with frequency 3 or more
vectorizer = CountVectorizer(min_df=2)
corpus_vect = vectorizer.fit(corpus)
X = corpus_vect.transform(corpus)

print("Shape of sparse matrix with corpus is:", X.shape)
print("Sample of the values is:\n", X[0:2])

#Vocabulary is already generated by the vectorizer
vocab = tuple(corpus_vect.vocabulary_.keys())

#Titles are the ID's of our instagram posts
titles = tuple(community_2_hashtags_df['ID'].to_numpy())

print("Example of vocabulary:", vocab[0:5])

print("Example of titles:", titles[0:5])

model = lda.LDA(n_topics=2, n_iter=3500, random_state=1)
model.fit(X.toarray())  # Fit the model
topic_word = model.topic_word_ 

INFO:lda:n_documents: 1499
INFO:lda:vocab_size: 109
INFO:lda:n_words: 4210
INFO:lda:n_topics: 2
INFO:lda:n_iter: 3500
INFO:lda:<0> log likelihood: -21511
INFO:lda:<10> log likelihood: -15477
INFO:lda:<20> log likelihood: -15176
INFO:lda:<30> log likelihood: -15226
INFO:lda:<40> log likelihood: -15182


Shape of sparse matrix with corpus is: (1499, 109)
Sample of the values is:
   (0, 21)	1
  (1, 63)	1
Example of vocabulary: ('crypto', 'next100xtoken', 'shiborgarmy', 'bluezilla', 'idos')
Example of titles: (0, 4, 6, 17, 30)


INFO:lda:<50> log likelihood: -15174
INFO:lda:<60> log likelihood: -15171
INFO:lda:<70> log likelihood: -15184
INFO:lda:<80> log likelihood: -15190
INFO:lda:<90> log likelihood: -15177
INFO:lda:<100> log likelihood: -15155
INFO:lda:<110> log likelihood: -15163
INFO:lda:<120> log likelihood: -15180
INFO:lda:<130> log likelihood: -15190
INFO:lda:<140> log likelihood: -15192
INFO:lda:<150> log likelihood: -15174
INFO:lda:<160> log likelihood: -15198
INFO:lda:<170> log likelihood: -15192
INFO:lda:<180> log likelihood: -15190
INFO:lda:<190> log likelihood: -15186
INFO:lda:<200> log likelihood: -15176
INFO:lda:<210> log likelihood: -15144
INFO:lda:<220> log likelihood: -15172
INFO:lda:<230> log likelihood: -15200
INFO:lda:<240> log likelihood: -15169
INFO:lda:<250> log likelihood: -15174
INFO:lda:<260> log likelihood: -15189
INFO:lda:<270> log likelihood: -15161
INFO:lda:<280> log likelihood: -15161
INFO:lda:<290> log likelihood: -15165
INFO:lda:<300> log likelihood: -15174
INFO:lda:<310> lo

In [ ]:
# Print top topic words 
n_top_words = 12
for i, topic_dist in enumerate(topic_word):
     topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
     print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: cryptone gateio nftcollectors cryptoinvestment wolverinu metaverse beyondprotocol shibarmy pixs nftart nftc bnb
Topic 1: metaverse anime solanaairdrop eth linksync shillme altcoin shiborgarmy cryptophyte ethereum bsc shibarmy


In [ ]:
df = pd.DataFrame(topic_word, columns = list(vocab))
#Sort by topic 0
df.transpose().sort_values([0], ascending = False).head(15)

,0,1
cryptone,0.148057,0.000005
gateio,0.069419,0.000005
nftcollectors,0.059226,0.000005
cryptoinvestment,0.051944,0.000005
wolverinu,0.047576,0.000005
metaverse,0.037867,0.165425
beyondprotocol,0.035440,0.000005
shibarmy,0.034469,0.018591
nftc,0.033013,0.000005
bnb,0.033013,0.000005


In [ ]:
#Sort by topic 1
df.transpose().sort_values([1], ascending = False).head(15)

,0,1
metaverse,0.037867,0.165425
anime,0.000005,0.152415
solanaairdrop,0.000005,0.107807
eth,0.000005,0.091079
linksync,0.000005,0.060411
shillme,0.000005,0.055764
altcoin,0.000005,0.043218
shiborgarmy,0.000005,0.031602
cryptophyte,0.000005,0.030208
ethereum,0.000005,0.028814


In [ ]:
#Sort by topic 2
df.transpose().sort_values([2], ascending = False).head(15)

,0,1,2
metaverse,0.035563,0.000008,0.270434
eth,0.000006,0.000008,0.140601
linksync,0.000006,0.000008,0.093258
shillme,0.000006,0.000008,0.086085
shiborgarmy,0.000006,0.000008,0.048785
cryptophyte,0.000006,0.000008,0.046633
shibarmy,0.044296,0.000008,0.028700
bluezilla,0.000006,0.000008,0.026548
ethereum,0.000006,0.020566,0.026548
shibelon,0.000006,0.000008,0.024396


## Community 3 (ETH)

In [ ]:
hashtags = community_3_tweets_df['text'].str.extractall(r'(#\S+)')
from collections import defaultdict
community_3_hashtags = defaultdict(str)
for index,row in hashtags.iterrows():
  print(index[0],row[0])
  community_3_hashtags[index[0]] += row[0].split('\\')[0] + " "

community_3_hashtags_df = pd.DataFrame(community_3_hashtags.items(), columns=['ID','labels'])
community_3_hashtags_df

59 #massadoption
59 #crypto'
85 #massadoption
85 #crypto'
100 #massadoption
100 #crypto'
122 #richquack
122 #quackattackistanbul
122 #quackattackturkey
122 #cryptocurrency
122 #crypto\xe2\x80\xa6'
166 #massadoption
166 #crypto'
184 #massadoption
184 #crypto'
198 #massadoption
198 #crypto'
209 #massadoption
209 #crypto'
224 #saitama
255 #massadoption
255 #crypto'
287 #hiphopgamer
287 #crypto
287 #gaming
287 #medical
287 #education
287 #saitamawolfpack
299 #starlink
299 #metaverse
299 #nft\xf0\x9f\x8c\x8f\xf0\x9f\x8c\x8f\xf0\x9f\x8c\x8f\xf0\x9f\x8c\x8f\xf0\x9f\x8c\x8f\xf0\x9f\x8c\x8f
299 #gamefi
299 #playtoearn\xf0\x9f\x92\xb5\xf0\x9f\x92\xb5\xf0\x9f\x92\xb5\xf0\x9f\x92\xb5\xf0\x9f\x92\xb5
299 #nft
299 #\xe2\x80\xa6'
316 #starlink
316 #metaverse
316 #nft\xf0\x9f\x8c\x8f\xf0\x9f\x8c\x8f\xf0\x9f\x8c\x8f\xf0\x9f\x8c\x8f\xf0\x9f\x8c\x8f\xf0\x9f\x8c\x8f
316 #gamefi
316 #playtoearn\xf0\x9f\x92\xb5\xf0\x9f\x92\xb5\xf0\x9f\x92\xb5\xf0\x9f\x92\xb5\xf0\x9f\x92\xb5
316 #nft
316 #\xe2\x80\xa6'
323 #

,ID,labels
0,59,#massadoption #crypto'
1,85,#massadoption #crypto'
2,100,#massadoption #crypto'
3,122,#richquack #quackattackistanbul #quackattacktu...
4,166,#massadoption #crypto'
...,...,...
335,11390,#elongate
336,11416,#fegex #sync
337,11451,#elongate
338,11461,#crypto'


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = list(community_3_hashtags_df['labels'].to_numpy())

#Vectorize, but consider only words with frequency 3 or more
vectorizer = CountVectorizer(min_df=2)
corpus_vect = vectorizer.fit(corpus)
X = corpus_vect.transform(corpus)

print("Shape of sparse matrix with corpus is:", X.shape)
print("Sample of the values is:\n", X[0:2])

#Vocabulary is already generated by the vectorizer
vocab = tuple(corpus_vect.vocabulary_.keys())

#Titles are the ID's of our instagram posts
titles = tuple(community_3_hashtags_df['ID'].to_numpy())

print("Example of vocabulary:", vocab[0:5])

print("Example of titles:", titles[0:5])

model = lda.LDA(n_topics=2, n_iter=3500, random_state=1)
model.fit(X.toarray())  # Fit the model
topic_word = model.topic_word_ 

INFO:lda:n_documents: 340
INFO:lda:vocab_size: 98
INFO:lda:n_words: 1056
INFO:lda:n_topics: 2
INFO:lda:n_iter: 3500
INFO:lda:<0> log likelihood: -5931
INFO:lda:<10> log likelihood: -4484
INFO:lda:<20> log likelihood: -4431
INFO:lda:<30> log likelihood: -4445
INFO:lda:<40> log likelihood: -4420
INFO:lda:<50> log likelihood: -4400


Shape of sparse matrix with corpus is: (340, 98)
Sample of the values is:
   (0, 21)	1
  (0, 59)	1
  (1, 21)	1
  (1, 59)	1
Example of vocabulary: ('massadoption', 'crypto', 'richquack', 'quackattackistanbul', 'quackattackturkey')
Example of titles: (59, 85, 100, 122, 166)


INFO:lda:<60> log likelihood: -4392
INFO:lda:<70> log likelihood: -4420
INFO:lda:<80> log likelihood: -4409
INFO:lda:<90> log likelihood: -4398
INFO:lda:<100> log likelihood: -4421
INFO:lda:<110> log likelihood: -4414
INFO:lda:<120> log likelihood: -4418
INFO:lda:<130> log likelihood: -4426
INFO:lda:<140> log likelihood: -4399
INFO:lda:<150> log likelihood: -4384
INFO:lda:<160> log likelihood: -4396
INFO:lda:<170> log likelihood: -4408
INFO:lda:<180> log likelihood: -4423
INFO:lda:<190> log likelihood: -4406
INFO:lda:<200> log likelihood: -4410
INFO:lda:<210> log likelihood: -4414
INFO:lda:<220> log likelihood: -4438
INFO:lda:<230> log likelihood: -4428
INFO:lda:<240> log likelihood: -4417
INFO:lda:<250> log likelihood: -4400
INFO:lda:<260> log likelihood: -4424
INFO:lda:<270> log likelihood: -4443
INFO:lda:<280> log likelihood: -4416
INFO:lda:<290> log likelihood: -4415
INFO:lda:<300> log likelihood: -4424
INFO:lda:<310> log likelihood: -4420
INFO:lda:<320> log likelihood: -4417
INFO:

In [ ]:
# Print top topic words 
n_top_words = 12
for i, topic_dist in enumerate(topic_word):
     topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
     print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: eth fegarmy xrp saitamatoken cryptocurrency family nftgaming invest smartlend gaming dec binanc
Topic 1: eth sync nfts crypt retweet doge btc education related rze cryprocurrency massadoption


In [ ]:
df = pd.DataFrame(topic_word, columns = list(vocab))
#Sort by topic 0
df.transpose().sort_values([0], ascending = False).head(15)

,0,1
eth,0.180597,0.210562
fegarmy,0.072855,0.000025
xrp,0.050093,0.000025
saitamatoken,0.047058,0.000025
cryptocurrency,0.044023,0.000025
family,0.042505,0.000025
nftgaming,0.033400,0.000025
invest,0.030365,0.000025
smartlend,0.027330,0.000025
gaming,0.025813,0.000025


In [ ]:
#Sort by topic 1
df.transpose().sort_values([1], ascending = False).head(15)

,0,1
eth,0.180597,0.210562
sync,0.012155,0.132864
nfts,0.000015,0.042634
doge,0.000015,0.040127
retweet,0.000015,0.040127
crypt,0.000015,0.040127
btc,0.000015,0.037621
education,0.019743,0.030102
related,0.000015,0.025089
rze,0.000015,0.022583


In [ ]:
#Sort by topic 2
df.transpose().sort_values([2], ascending = False).head(15)

,0,1,2
eth,0.304850,0.141989,0.159768
sync,0.106206,0.000021,0.104209
nfts,0.000034,0.000021,0.059067
retweet,0.000034,0.000021,0.055594
crypt,0.000034,0.000021,0.055594
btc,0.000034,0.000021,0.052122
education,0.044558,0.000021,0.041704
related,0.000034,0.000021,0.034759
massadoption,0.000034,0.000021,0.027814
metaplay,0.000034,0.000021,0.027814


## Topics by Titles

In [ ]:
doc_topic = model.doc_topic_
print("Post ID \t Adventure \t Photoshoot \t Clothes")
for i in range(201):
    print(titles[i], "\t\t", np.round(doc_topic[i][0],9), "\t", np.round(doc_topic[i][1],9), "\t", np.round(doc_topic[i][2],9) )

Post ID 	 Adventure 	 Photoshoot 	 Clothes
0 		 0.018867925 	 0.962264151 	 0.018867925
4 		 0.076923077 	 0.846153846 	 0.076923077
6 		 0.076923077 	 0.846153846 	 0.076923077
17 		 0.018867925 	 0.962264151 	 0.018867925
30 		 0.015873016 	 0.968253968 	 0.015873016
41 		 0.076923077 	 0.846153846 	 0.076923077
56 		 0.076923077 	 0.846153846 	 0.076923077
67 		 0.018867925 	 0.018867925 	 0.962264151
75 		 0.043478261 	 0.913043478 	 0.043478261
77 		 0.097345133 	 0.628318584 	 0.274336283
78 		 0.018867925 	 0.018867925 	 0.962264151
80 		 0.043478261 	 0.913043478 	 0.043478261
91 		 0.076923077 	 0.846153846 	 0.076923077
92 		 0.076923077 	 0.846153846 	 0.076923077
98 		 0.018867925 	 0.962264151 	 0.018867925
104 		 0.018867925 	 0.962264151 	 0.018867925
106 		 0.076923077 	 0.846153846 	 0.076923077
109 		 0.043478261 	 0.913043478 	 0.043478261
131 		 0.01369863 	 0.01369863 	 0.97260274
139 		 0.846153846 	 0.076923077 	 0.076923077
148 		 0.076923077 	 0.846153846 	 0.0

# Stuff

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
import numpy as np

wordcount_df = pd.read_csv('11534_tweets.csv')

In [ ]:
# wordcount_df.sort_values(by="Count", ascending=False)
wordcount_df

,id,name,screen_name,created_at,text,location,followers_count,friends_count,listed_count,retweet_count,favorite_count
0,1460631062023917570,Lucas Stollmeier,StollmeierLucas,2021-11-16 15:29:17,b'RT @SheldonEvans: Just hit 200K!\n\xf0\x9f\x...,NaN,0,22,0,4702,0
1,1460631061038198796,علی حیدری,lyhydry1,2021-11-16 15:29:16,b'RT @heliosarmy: @lexfridman We believe that ...,NaN,3,3,0,84,0
2,1460631060778303490,Coldstar,Coldsta93422887,2021-11-16 15:29:16,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,5,346,1,1318,0
3,1460631059889115149,PJ,Jose_1017,2021-11-16 15:29:16,b'@davidgokhshtein Load your bags we\xe2\x80\x...,Puerto Rico,55,581,2,0,0
4,1460631057636679680,ام سوسو,omsosoo0,2021-11-16 15:29:16,b'RT @Next100xToken: \xf0\x9f\x94\xa5 #Next100...,NaN,84,4340,0,520,0
...,...,...,...,...,...,...,...,...,...,...,...
11529,1461103726064812032,Muhammad Asad Nadeem,Muhammad8255011,2021-11-17 22:47:28,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,7,111,0,18092,0
11530,1461103722126323716,Benedito Campos,Benedit80592865,2021-11-17 22:47:28,b'RT @AIBCsummit: A huge thank you to everyone...,NaN,26,63,1,1544,0
11531,1461103721388167172,Yen Ngoc,YenNgoc45,2021-11-17 22:47:27,b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...,NaN,0,22,0,4495,0
11532,1461103717265063939,Billy - Crypto 💻🔑,billydavies___,2021-11-17 22:47:26,b'The market bleeeeds \xf0\x9f\xa9\xb8 again t...,Australia,64,221,0,0,0


In [ ]:
a_11534_tweets = pd.read_csv('11534_tweets.csv')['text']
a_11534_tweets

0        b'RT @SheldonEvans: Just hit 200K!\n\xf0\x9f\x...
1        b'RT @heliosarmy: @lexfridman We believe that ...
2        b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...
3        b'@davidgokhshtein Load your bags we\xe2\x80\x...
4        b'RT @Next100xToken: \xf0\x9f\x94\xa5 #Next100...
                               ...                        
11529    b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...
11530    b'RT @AIBCsummit: A huge thank you to everyone...
11531    b'RT @CryptoTownEU: \xf0\x9f\x9a\x80 Airdrop: ...
11532    b'The market bleeeeds \xf0\x9f\xa9\xb8 again t...
11533    b'RT @revolut20: Thank you to @CryptonationN f...
Name: text, Length: 11534, dtype: object

In [ ]:
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", \
             "ourselves", "you", "your", "yours", "yourself", "yourselves", \
             "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", \
             "its", "itself", "they", "them", "their", "theirs", "themselves", "what", \
             "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", \
             "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", \
             "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", \
             "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", \
             "between", "into", "through", "during", "before", "after", "above", "below", \
             "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", \
             "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", \
             "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", \
             "not", "only", "own", "same", "so", "than", "too", "very", "t", "can", "will", \
             "just", "don", "should", "now", "=", "et", "The", "al.,", "In", "We", "+", "-", \
             "&", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "−", "", "∈", "\t", "cid", \
             "al", "q", "w", "e", "r", "t", "y", "u", "i", "o", "p", "a", "s", "d", "f", "g", \
             "h", "j", "k", "l", "z", "x", "c", "v", "b", "n", "m", "µ", "re", "ct", "αk", \
             "dp", "l0", "l1", "l2", "yt", "xt", "σ", "ms", "λi", "aij"]

In [ ]:
list_of_unique_words = []

for rows in a_11534_tweets:
  for words in str(rows).split(" "):
    # Checking 3 conditions:
    # 1. the word is not already in the canonical list of words
    # 2. the word length is greater than 3
    # 3. the word is not a stopword
    if not (str(words) in list_of_unique_words) and len(str(words)) >= 3 and not (str(words) in stopwords):
      list_of_unique_words.append(str(words.split('\\')[0]))

list_of_unique_words

["b'RT",
 '@SheldonEvans:',
 'Just',
 'hit',
 '200K!',
 'AWAY',
 '$1,000',
 '#crypto',
 'Like',
 'Retweet',
 'This',
 'Follow',
 'Me',
 'friends',
 'shares',
 'i',
 '@heliosarmy:',
 '@lexfridman',
 'believe',
 '#Space',
 'every',
 'one.',
 '#Helios',
 'space',
 'beyond.',
 '',
 'review',
 '@missionhelios',
 'p',
 '@CryptoTownEU:',
 '',
 'Airdrop:',
 'Adora',
 'Value:',
 'worth',
 '$ARA',
 'Referral:',
 '100',
 '$ARA',
 'Exchange:',
 'CoinmarketCap,',
 'CoinGecko,',
 'BigOne,',
 'Bico',
 "b'@davidgokhshtein",
 'Load',
 'bags',
 'we',
 'getting',
 'ready',
 'take',
 'off.',
 '#occt',
 '#bullrun',
 '#community',
 '#1000x',
 '#crypto.',
 "https://t.co/kCHXAKFOYO'",
 '@Next100xToken:',
 '',
 '#Next100xToken',
 '',
 'For',
 'Any',
 'Promotion',
 '------',
 'Join',
 'Our',
 'Most',
 'Active',
 'Network',
 '70K+',
 'Telegram',
 '$Next100xToken',
 'Chan',
 "b'",
 'Review',
 'Curated',
 'post',
 'Publish0x',
 'Read',
 'write',
 'articles',
 'get',
 '',
 '#referral',
 '#earn',
 '#btc',
 '@Shiborg

In [ ]:
# This cell 3 takes minutes to complete (or more)
list_of_appearances_words = []
list_of_unique_appearances_words = []

# Counting << ONLY 1 >> the appearance of a word in the message post
for words in list_of_unique_words:
  count = 0
  for rows in a_11534_tweets:
    if words in str(rows).split(" "):
      count += 1

  list_of_unique_appearances_words.append(count)

In [ ]:
dict_unique_appearances_per_post = dict(zip(list_of_unique_words, list_of_unique_appearances_words))
dict_unique_appearances_per_post

{"b'RT": 8242,
 '@SheldonEvans:': 133,
 'Just': 182,
 'hit': 162,
 '200K!': 0,
 'AWAY': 134,
 '$1,000': 159,
 '#crypto': 1113,
 'Like': 596,
 'Retweet': 543,
 'This': 471,
 'Follow': 535,
 'Me': 10,
 'friends': 156,
 'shares': 137,
 'i': 43,
 '@heliosarmy:': 1,
 '@lexfridman': 1,
 'believe': 69,
 '#Space': 6,
 'every': 64,
 'one.': 3,
 '#Helios': 13,
 'space': 27,
 'beyond.': 1,
 '': 867,
 'review': 8,
 '@missionhelios': 1,
 'p': 0,
 '@CryptoTownEU:': 1551,
 'Airdrop:': 1442,
 'Adora': 0,
 'Value:': 1554,
 'worth': 362,
 '$ARA': 0,
 'Referral:': 1546,
 '100': 450,
 'Exchange:': 563,
 'CoinmarketCap,': 181,
 'CoinGecko,': 299,
 'BigOne,': 299,
 'Bico': 0,
 "b'@davidgokhshtein": 5,
 'Load': 3,
 'bags': 8,
 'we': 386,
 'getting': 193,
 'ready': 134,
 'take': 62,
 'off.': 3,
 '#occt': 4,
 '#bullrun': 10,
 '#community': 6,
 '#1000x': 3,
 '#crypto.': 11,
 "https://t.co/kCHXAKFOYO'": 1,
 '@Next100xToken:': 73,
 '#Next100xToken': 73,
 'For': 111,
 'Any': 76,
 'Promotion': 73,
 '------': 0,
 'J

In [ ]:
sorted(dict_unique_appearances_per_post.items(), key = lambda kv: kv[1], reverse=True)

[("b'RT", 8242),
 ('the', 3635),
 ('to', 2953),
 ('is', 2484),
 ('and', 2294),
 ('of', 1951),
 ('on', 1898),
 ('a', 1665),
 ('Value:', 1554),
 ('@CryptoTownEU:', 1551),
 ('Referral:', 1546),
 ('Airdrop:', 1442),
 ('in', 1371),
 ('for', 1337),
 ('you', 1154),
 ('#crypto', 1113),
 ('this', 1055),
 ('I', 1024),
 ('#Crypto', 1000),
 ('$', 976),
 ('will', 909),
 ('b"RT', 895),
 ('&amp;', 884),
 ('with', 872),
 ('', 867),
 ('be', 840),
 ('Date:', 704),
 ('End', 703),
 ('at', 665),
 ('your', 614),
 ('are', 612),
 ('Like', 596),
 ('New', 588),
 ('our', 581),
 ('Audit:', 580),
 ('Exchange:', 563),
 ('Retweet', 543),
 ('Follow', 535),
 ('#Airdrop:', 525),
 ('1', 518),
 ('it', 483),
 ('Top', 477),
 ('This', 471),
 ('-', 462),
 ('100', 450),
 ('@revolut20:', 449),
 ('project', 439),
 ('Start', 432),
 ('that', 431),
 ('Worth', 424),
 ('airdrop', 421),
 ('bot', 407),
 ('@AirdropDet:', 406),
 ('look', 392),
 ('has', 387),
 ('we', 386),
 ('about', 386),
 ('referral', 384),
 ('worth', 362),
 ('always',

In [ ]:
dict_appearances_per_post = dict(zip(list_of_unique_words, list_of_appearances_words))
dict_appearances_per_post

{}

In [ ]:
# This cell 3 takes minutes to complete (or more)
list_of_appearances_words = []
list_of_unique_appearances_words = []
word_a_list =  ['NFT', '#btc', '#bitcoin', '#Crypto', "#eth", "#nft", '#metaverse', '#meta', '#nftcollectors']
word_b_list = ['Airdrop','#Airdrop',"Winners",'future', 'good', 'best', 'super', 'Moonsheet', '\\xf0\\x9f\\x9a\\x80', '#bullish']
similarity_df = pd.DataFrame(0.0, index=word_a_list, columns=word_b_list)
# Counting << ONLY 1 >> the appearance of a word in the message post
for word_a in word_a_list:
  #print("\n", word_a)
  for word_b in word_b_list:
    count_a = 0
    count_b = 0
    count_ab = 0
    count_n = 0
    for rows in a_11534_tweets:
      count_n += 1
      if word_a in str(rows).split(" "):
        if word_b in str(rows).split(" "):
          count_ab += 1
          count_a += 1
          count_b += 1
        else:
          count_a += 1
      else:
        if word_b in str(rows).split(" "):
          count_b += 1


    if count_a == 0 or count_b == 0:
      lift_value = 0
    else:
      lift_value = 11534*count_ab/(count_a*count_b)
    #print(word_b, "\t", 11534*count_ab/(count_a*count_b))
    similarity_df.at[word_a, word_b] = lift_value

similarity_df

,Airdrop,#Airdrop,Winners,future,good,best,super,Moonsheet,\xf0\x9f\x9a\x80,#bullish
NFT,24.141591,20.995537,0.0,0.000000,0.235417,0.000000,0.000000,0.575463,0.027670,0.000000
#btc,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
#bitcoin,1.617672,0.000000,0.0,0.511713,0.000000,3.774568,0.000000,0.000000,0.126079,0.000000
#Crypto,0.000000,6.033169,0.0,0.235388,1.110225,0.310054,0.067450,0.000000,0.021749,7.689333
#eth,0.000000,0.000000,0.0,0.000000,0.000000,1.771736,0.000000,0.000000,0.000000,0.000000
#nft,0.000000,0.000000,0.0,0.000000,0.000000,20.175747,0.322729,0.000000,0.000000,0.000000
#metaverse,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.705771,0.000000
#meta,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
#nftcollectors,0.000000,0.000000,0.0,0.000000,0.000000,58.565711,0.000000,0.000000,0.000000,0.000000


In [ ]:
# Sorting dictionary << dict_unique_appearances_per_post >>
wordz = dict(sorted(dict_unique_appearances_per_post.items(), key=lambda item: item[1], reverse=True))

In [ ]:
count = 10
for word in wordz:
  if count <1:
    break
  print (word)
  count-=1

In [ ]:

from sklearn.manifold import MDS
from collections import defaultdict

# 4) Python list of top attributes (We hand picked these attributes)
top_attributes = ["bull", "bear", "crypto", "airdrop", "nft", "scam", "wallet", "mining", "defi", "mint", "staking", "finance", "future", "revolution"]

In [ ]:
### Definition of functions

#---------------------------------------------------------------------------
# Define functions to find and count messages
#
# Note: The dataframes are expected to have 2 columns: ['Date', 'Messages']
#---------------------------------------------------------------------------

def find_messages_with_str(df, *argv):
  """This function takes a dataframe = df, and a any number of word parameters 
  through the *argv arguments. This function returns a NEW dataframe with the
  messages that contain all the words (word1 & word2 & ... wordn) unwrapped 
  from the *argvs

  Parameters:
         df = Dataframe with the columns ['Date', 'Messages']
      *argv = Every argument is a different word to be matched
  """

  #Create a regular expression pattern containing all words
  regex_pattern = ''
  for arg in argv:
    regex_pattern += '(?=.*' + arg + ')'

  #filter the dataframe through the regular expression and return the new df
  result = df['Messages'].str.contains(pat = regex_pattern, regex=True)
  filtered_df = df.loc[result]
  return filtered_df


def find_messages_with_any_str(df, *argv):
  """This function takes a dataframe = df, and a any number of word parameters 
  through the *argv arguments. This function returns a NEW dataframe with the
  messages that contain ANY the words (word1 | word2 | ...| wordn) unwrapped 
  from the *argvs (This is an OR operator)

  Parameters:
         df = Dataframe with the columns ['Date', 'Messages']
      *argv = Every argument is a different word to be matched
  """

  #Create a regular expression pattern containing all words
  regex_pattern = ''
  for arg in argv:
    regex_pattern += '(?=.*' + arg + ')|'
  regex_pattern = regex_pattern[0:-1]
  
  #filter the dataframe through the regular expression and return the new df
  result = df['Messages'].str.contains(pat = regex_pattern, regex=True)
  filtered_df = df.loc[result]
  return filtered_df


def count_messages(df):
  """This is a simple function to count messages in a dataframe
  It makes sense to use it with the dataframe containing ONLY 
  the messages that are relevant to the count
  
  Parameters:
         df = Dataframe with the columns ['Date', 'Messages']
  """
  
  return df['Messages'].count()


def count_messages_with_word(df, *argv):
  """This function look for the messages with the words in *argv, 
  then counts the resulting dataframe to provide the count of the
  messages that have a combination of words.
  
  Parameters:
         df = Dataframe with the columns ['Date', 'Messages']
      *argv = Every argument is a different word to be matched

  """

  new_df = find_messages_with_str(df, *argv)
  return count_messages(new_df)

def calc_lift_vals(num_msgs_a, num_msgs_b, num_msgs_ab, total_msgs):
  """This function calculates the lift values. It takes four arguments: 
  the number messages in which the first word appears, the number of 
  messages in which the second word appears, the total number of messages
  in which both words appear, and the total number of messages overall.
  """

  lift_val = (total_msgs * num_msgs_ab) / (num_msgs_a  *num_msgs_b)
  return lift_val

In [ ]:
top_brands = ["eth", "", ""]

# Count the number of messages (Sample size)
N = count_messages(data_df)
print("The number of messages is: ", N)

# Count the number of messages that mention each brand from top_brands
# (count each message only once)
messages_dict_per_brand = defaultdict(dict)
for brand in top_brands:
  messages_dict_per_brand[brand][brand] = count_messages_with_word(data_df, brand)

# Count the number of messages that mention both X and Y in the same message,
#where X and Y are both pairwise brands from top_brand 
for brand_1 in top_brands:
    for brand_2 in top_brands:
      if brand_1 == brand_2:
        continue
      messages_dict_per_brand[brand_1][brand_2] = count_messages_with_word(data_df, brand_1, brand_2)

#Counts generated
print("The pairwise mentions are:\n", messages_dict_per_brand)